In [1]:
import pandas as pd
#import boto3
#import anthropic_bedrock
#from anthropic_bedrock import AnthropicBedrock
from tqdm import tqdm
import hashlib
import argparse
import csv
import re
import json
import configparser
import os
import numpy as np
from vllm import LLM, SamplingParams

from tqdm import tqdm
tqdm.pandas()

In [2]:
all_qac=pd.read_csv('all_qac_triplets.csv').drop_duplicates(subset=['original_ques','context','filename'])[['original_ques','context','filename']]
#all_qac=pd.read_csv('all_qac_triplets.csv')
all_qac.head(5)

,original_ques,context,filename
0,What type of ECG will be performed at screening?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...
1,How often will the ECG be performed?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...
2,What procedures will be used to perform the ECG?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...
3,What should be done with clinically significan...,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...
4,Where should clinically significant abnormal E...,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...


In [3]:
all_qac['ID'] = all_qac['filename'].apply(lambda x: x.split("/")[-1].replace(".txt",""))
all_qac.head()

,original_ques,context,filename,ID
0,What type of ECG will be performed at screening?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62
1,How often will the ECG be performed?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62
2,What procedures will be used to perform the ECG?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62
3,What should be done with clinically significan...,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62
4,Where should clinically significant abnormal E...,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62


In [4]:
context_id = all_qac.drop_duplicates(subset=['context','ID'])
unique_context = context_id["context"].tolist()
unique_id = context_id['ID'].tolist()

In [5]:
print(len(unique_context),len(unique_id))

15618 15618


In [6]:
context_id.head()

,original_ques,context,filename,ID
0,What type of ECG will be performed at screening?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62
5,What is required if there are interruptions fr...,7.1.1.9 Trial Compliance (Medication/Diet/Acti...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_58
9,Who will review the inclusion and exclusion cr...,7.1.1.2 Inclusion/Exclusion Criteria\nAllinclu...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_51
22,What are subjects excluded for regarding prior...,5.1.3 Subject Exclusion Criteria\nThe subject ...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_24
36,What fixed dose of pembrolizumab will be used ...,5.2.1.1 Dose Selection (Preparation)\nThe rati...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_26


In [7]:
context_id['doc_path'] = context_id['filename'].apply(lambda x: '/'.join(z for z in x.split("/")[3:-1]))
context_id.head()

/tmp/ipykernel_2135617/1272513008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_id['doc_path'] = context_id['filename'].apply(lambda x: '/'.join(z for z in x.split("/")[3:-1]))


,original_ques,context,filename,ID,doc_path
0,What type of ECG will be performed at screening?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62,NSCLC-Protocol/15042024/NCT02142738
5,What is required if there are interruptions fr...,7.1.1.9 Trial Compliance (Medication/Diet/Acti...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_58,NSCLC-Protocol/15042024/NCT02142738
9,Who will review the inclusion and exclusion cr...,7.1.1.2 Inclusion/Exclusion Criteria\nAllinclu...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_51,NSCLC-Protocol/15042024/NCT02142738
22,What are subjects excluded for regarding prior...,5.1.3 Subject Exclusion Criteria\nThe subject ...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_24,NSCLC-Protocol/15042024/NCT02142738
36,What fixed dose of pembrolizumab will be used ...,5.2.1.1 Dose Selection (Preparation)\nThe rati...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_26,NSCLC-Protocol/15042024/NCT02142738


In [ ]:
#Making a dict for all the documents and the corresponding questions
qac_count={}


for i, row in all_qac.iterrows():
    
    if(i%100000==0):
        print(i)
        #print(row['doc_path'])
    doc=row['filename']
    if doc in qac_count.keys():
        val=qac_count[doc][-1]
        val+=1
        qac_count[doc][-1]=val
        qac_count[doc][0].append(row['original_ques'])
       
    else:
        qac_count[doc]=[[row['original_ques']],1]
        #print( qac_count[doc][0])
    
qac_count       

In [9]:
print(len(qac_count))
print(len(all_qac))

15896
505009


In [10]:
sum=0
max=0
max_key=''
for key,value in qac_count.items():
    if qac_count[key][-1]>max:
        max=qac_count[key][-1]
        max_key=key
    sum=sum+qac_count[key][-1]
print(max,sum,max_key)

144 505009 CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Paper/20240315_00_v2/NCT00735878_000343165/page_18.txt


In [11]:
all_qac['doc_path'] = all_qac['filename'].apply(lambda x: '/'.join(z for z in x.split("/")[3:-1]))
all_qac.head()

,original_ques,context,filename,ID,doc_path
0,What type of ECG will be performed at screening?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62,NSCLC-Protocol/15042024/NCT02142738
1,How often will the ECG be performed?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62,NSCLC-Protocol/15042024/NCT02142738
2,What procedures will be used to perform the ECG?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62,NSCLC-Protocol/15042024/NCT02142738
3,What should be done with clinically significan...,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62,NSCLC-Protocol/15042024/NCT02142738
4,Where should clinically significant abnormal E...,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,CT-Documents-Hub/50-CQAGen/Pending/NSCLC-Proto...,page_62,NSCLC-Protocol/15042024/NCT02142738


In [ ]:
qac_count_doc={}


for i, row in all_qac.iterrows():
    
    if(i%100000==0):
        print(i)
        
    doc=row['doc_path']
    if doc in qac_count_doc.keys():
        val=qac_count_doc[doc][-1]
        val+=1
        qac_count_doc[doc][-1]=val
        qac_count_doc[doc][0].append(row['original_ques'])
       
    else:
        qac_count_doc[doc]=[[row['original_ques']],1]
        
    
qac_count_doc       

In [14]:
print(len(qac_count_doc))


1144


In [15]:
#Prompt for question rephrasing
def prompt1(alltext):
    return f'''I'm going to giving you the questions generated from a clinical trial protocol document. Then I'm going to ask you to generate at most two rephrased questions for every given question. Here are the questions:

<questions>

{alltext}

</questions>

Generate the rephrased versions of the questions. Make sure you rephrase only the questions given within the tags.

Each question and its rephrased versions should follow this format:
Q1: Original question.
Q1.1: First rephrased question.
Q1.2: Second rephrased question.

To create useful paraphrases, avoid using the same 'question' word as in the original question. For example, if the original question is 'What is the patient's age?' the paraphrase could be 'How old is the patient?' or 'How many years ago was the patient born?' or 'How long ago was the patient born?' etc.

The format of your overall response should match what's shown between the tags. Ensure to follow the formatting and spacing exactly.

<example>

Q1: What safety measures are in place for participants?
Q1.1: What precautions are taken to ensure participant safety?
Q1.2: How is participant safety maintained during the study?

Q2: How long is the duration of the study?
Q2.1: What is the total length of the study?
Q2.2: For what period will the study run?

Q3: What are the inclusion criteria for participants?
Q3.1: Which conditions must participants meet to be included?
Q3.2: Who is eligible to join this study?

</example>

Provide at most two rephrased questions for every question. Ensure that only the given questions and their rephrased versions are included in the output in the format shown in the example. The output should strictly follow the provided format, and the rephrased questions must not deviate significantly from the original question.'''


In [16]:
def system_prompt(prompt):
    #https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/
    return f'''<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant. Your carefully follow instructions.<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'''

In [ ]:
#Gnenerating the prompts by selecting random valid questions
import random
prompts=[]
k=0
not_ten=[]  
for key,val in qac_count_doc.items():
  questions=val[0]
  flag=0  
  valid_questions=set()    
  if(len(set(questions))<10):
      flag=1
      valid_questions=set(questions)
  
  while(len(valid_questions)!=10 and flag==0):
      valid_questions=set()
      #print(len(valid_questions),key)
      ques_rephrase = random.choices(questions, k=10)
      
      for question in ques_rephrase:
         #print(question) 
         if question.strip().endswith('?') and len(question.strip()) <= 300:
                valid_questions.add(question)  
  #text=f"Context:\n{context}\nQuestions:\n"
    
  text=''  
    
 

  valid_questions=list(set(valid_questions))
  
  if(len(valid_questions)!=10):
      #print(len(valid_questions),key)  
      not_ten.append(key)
  for i in range(len(valid_questions)):
        text=text+f"Q{i+1}:{valid_questions[i]}\n"
  #print(text)    
  prompt=system_prompt(prompt1(text))
  prompts.append(prompt)    

  
print(len(prompts))      


In [121]:
#Generating the output
llm = LLM(model="/lockbox/models/Meta-Llama-3-8B-Instruct", gpu_memory_utilization = 0.8)
sampling_params = SamplingParams(temperature=0.05, top_p=0.95, top_k = 40, max_tokens = 4816,)
out=[]
try:
    generated_outputs = llm.generate(prompts, sampling_params)
    for output in generated_outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        out.append(generated_text) 
except Exception:
            print(Exception)  


INFO 07-04 10:13:10 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='/lockbox/models/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='/lockbox/models/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/lockbox/models/Meta-Llama-3-8B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-04 10:13:31 model_runner.py:160] Loading model weights took 14.9575 GB
INFO 07-04 10:13:32 gpu_executor.py:83] # GPU blocks: 23966, # CPU blocks: 2048
INFO 07-04 10:13:33 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-04 10:13:33 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-04 10:13:47 model_runner.py:965] Graph capturing finished in 13 secs.


Processed prompts: 100%|██| 1144/1144 [04:13<00:00,  4.51it/s, est. speed input: 2565.14 toks/s, output: 2256.99 toks/s]


In [127]:
#writing the output to text file
for i in range(len(out)):
    temp=out[i]
    filename=f'./Test_Documents/all_rephrased_questions/out{i}.txt'
    with open(filename,'w') as f:
        f.write(temp)

In [85]:
with open('./Test_Documents/all_rephrased_questions/out195.txt','r') as f:
    print(f.read())
    



Here are the rephrased questions:

<questions>

Q1: How often are tumor markers and chest CT evaluated in the first 2 years?
Q1.1: What is the frequency of tumor marker and chest CT assessments during the first 2 years?
Q1.2: How frequently will tumor markers and chest CT scans be monitored during the initial 2-year period?

Q2: How many patients are expected to enroll from Tangdu Hospital?
Q2.1: What is the anticipated number of patients from Tangdu Hospital who will participate in the study?
Q2.2: How many patients are projected to be recruited from Tangdu Hospital?

Q3: What are two of the inclusion criteria?
Q3.1: Which two conditions must participants meet to be eligible for the study?
Q3.2: What are the two primary requirements for participant inclusion?

Q4: What are the secondary endpoints of the study?
Q4.1: What are the additional outcomes being measured in the study?
Q4.2: What are the secondary objectives of the research?

Q5: Where will the study findings be disseminated

In [ ]:
#extract question from text files
import re
import json
not_ten_file=[]
def extract_questions_from_file(file_path):
    questions = []
    current_question = None

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Match the original question
            original_match = re.match(r'(Q\d+):\s*(.*)', line)
            if original_match:
                current_question = {
                    "original": original_match.group(2),
                    "rephrased": []
                }
                questions.append(current_question)
                continue
            
            # Match the rephrased questions
            rephrased_match = re.match(r'(Q\d+\.\d+):\s*(.*)', line)
            if rephrased_match and current_question:
                current_question["rephrased"].append(rephrased_match.group(2))

    return questions

def extract_questions_from_files(file_paths):
    all_questions = []
    all_doc=list(qac_count_doc.keys())
    for file_path in file_paths:
        questions = extract_questions_from_file(file_path)
        if(len(questions)!=10):
            
            index=int(file_path.split('/')[3].replace(".txt", "").replace('out',""))
            if(all_doc[index]=='CART-Paper/20240307_00_v2/NCT01105247_blood810044-suppl1'):
                print(index)
            not_ten_file.append(all_doc[index])
            
            
        all_questions.extend(questions)
    return all_questions

def save_to_json(data, output_file):
    with open(output_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Path to the input file
path = './Test_Documents/all_rephrased_questions/'
all_files_names=os.listdir(path)
all_files=[path+file_name for file_name in all_files_names if '.ipynb_checkpoints' not in file_name ]
#print(all_files)
# Extract questions

questions_data = extract_questions_from_files(all_files)

# Save to JSON
#save_to_json(questions_data, 'questions.json')

print("Questions and their rephrased versions have been extracted and saved to questions.json")
#print(not_ten_file)
questions_data

In [ ]:
#Remove similar questions based on rouge score
def calculate_rouge_similarity(question1, question2):
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scores = scorer.score(question1, question2)
    return scores['rouge1'].fmeasure

def filter_similar_questions(questions_data, similarity_threshold=0.9):
    filtered_questions = []
    num_questions = len(questions_data)

    # Compare each question with every other question
    for i in range(num_questions):
        include = True
        for j in range(i + 1, num_questions):
            q1 = questions_data[i]['original']
            q2 = questions_data[j]['original']
            similarity = calculate_rouge_similarity(q1, q2)
            if similarity > similarity_threshold:
                print(q1,' ',q2)
                include = False
                break
        if include:
            filtered_questions.append(questions_data[i])

    return filtered_questions


filtered_questions = filter_similar_questions(questions_data)

In [64]:
with open('rephrased_questions_cleaned.json', 'w') as f:
    json.dump(filtered_questions, f, indent=2)
print(len(filtered_questions))

9873


In [69]:
with open('rephrased_questions_cleaned.json', 'r') as f:
    filtered_questions=json.load(f)
print(len(filtered_questions))

9873


In [90]:
#add the associated context and make a json
filtered_questions_with_context = []
for temp in filtered_questions:
    temp=temp.copy()
    ques=temp['original']
    #print(temp)
    index=all_qac[all_qac['original_ques'] == ques].index.tolist()
    if(len(index)==0):
        print(temp)
        continue
    index=index[0]    
    context=all_qac['context'][index]
    temp['context']=context
    filtered_questions_with_context.append(temp)

print(filtered_questions_with_context)

{'original': 'How frequently will tumor markers and chest CT scans be assessed during follow-up?', 'rephrased': ['How often will tumor markers and chest CT scans be monitored during the follow-up period?', 'What is the frequency of tumor marker and chest CT assessments during the follow-up phase?']}
{'original': 'What does the case demonstrate regarding real-time cytokine monitoring?', 'rephrased': ['What insights does the case provide about real-time cytokine monitoring?', 'How does the case illustrate the importance of real-time cytokine monitoring?']}
{'original': 'How did the amounts of intravenous lidocaine, magnesium sulfate, and epidural ropivacaine compare between groups?', 'rephrased': ['What were the differences in the amounts of intravenous lidocaine, magnesium sulfate, and epidural ropivacaine between treatment groups?', 'How did the dosages of intravenous lidocaine, magnesium sulfate, and epidural ropivacaine vary between the groups?']}
{'original': 'What does this study e

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [92]:
with open('rephrased_questions_cleaned_with_context.json', 'w') as f:
    json.dump(filtered_questions_with_context, f, indent=2)
print(len(filtered_questions_with_context))

9859
